In [18]:
import pandas as pd
import os
import os.path as osp

In [26]:
markdown = ''

companies_dir = '../companies'
for fname in os.listdir(companies_dir):
    fpath = osp.join(companies_dir, fname)
    company_name = fname.split('.')[0]
    df = pd.read_csv(fpath)
    if len(df.index) > 5:
        markdown += f'\n\n## {company_name}\n'
        markdown += df.to_markdown(tablefmt="grid", index=False)

In [28]:
with open('README.md', 'w') as f:
    f.write(markdown)